## 데이터 전처리

In [ ]:
%matplotlib inline

data = pd.read_csv("data0703.csv")
data = data.iloc[:,1:]

from sklearn.cluster import KMeans
region1 = data[["latitude", "longitude"]]
model = KMeans(n_clusters=10).fit(region1)
data["cluster"] = model.labels_

del data["fips"]
del data["latitude"]
del data["longitude"]
del data["taxvaluedollarcnt"]
del data["landtaxvaluedollarcnt"]
del data["taxamount"]

data = data[data["unitcnt"]==1]
data = data[data["bedroomcnt"]!=0]
data = data[data["bathroomcnt"]!=0]
del data["unitcnt"]

data.columns = ["aircondition", "bath", "bed", "qualitytype", "cfsf", "heating", "lssf", "spa", "pool", "landuse", "unitcnt", "year", "price","cluster"]
data = data[["aircondition", "bath", "bed", "qualitytype", "cfsf", "heating", "lssf", "spa", "pool", "landuse", "unitcnt", "year","cluster", "price"]]
#dfy = data["price"]
#dfX = data[["aircondition", "bath", "bed", "qualitytype", "cfsf", "heating", "lssf", "spa", "pool", "landuse", "unitcnt", "year", "cluster"]]

## 아웃라이어 제거 (2.5, 97.5)

In [ ]:
def outlier1(data, feature):
    low = np.percentile(data.feature, 2.5)
    up = np.percentile(data.feature, 97.5)

    data = data[(data.feature > low) & (data.feature < up)]
    return data

## 레버리지와 아웃라이어

In [5]:
def outlier(unit_data, n):
    num = n
    unit_data2 = unit_data
    for i in range(num):
        model = sm.OLS.from_formula("price ~ pc1 + pc2 + pc4 + pc6 + C(landuse) + C(cluster)", data=unit_data2).fit()
        print(model.summary())

        influence = model.get_influence()
        cooks_d2, pvals = influence.cooks_distance
        fox_cr = 4 / (len(unit_data2) - model.df_model - 1)
        idx = np.where(cooks_d2 > fox_cr)[0]
        
        print(str(i)+"번째")
        print(len(idx))
        print("")
        
        list(idx)
        i = np.arange(len(unit_data2))
        unit_data2 = unit_data2.set_index(i)
        unit_data2 = unit_data2.drop(idx)
        unit_data2.shape    
    return unit_data2

## correlation matrix

In [1]:
def corrmatrix(df):
    corrmat = df.corr()
    plt.subplots(figsize=(11, 9))
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    sns.heatmap(corrmat, vmax=.8, square=True, cmap=cmap)

## 지역별 클러스터링

In [2]:
from sklearn.cluster import KMeans
def clustering(data):
    region1 = data[["latitude", "longitude"]]
    model = KMeans(n_clusters=10).fit(region1)
    data["cluster"] = model.labels_
    return data

## 표준화

In [3]:
#독립변수들 표준화 
from sklearn.preprocessing import StandardScaler
def std(data):
    numeric_columns = ["aircondition", "bath", "bed", "qualitytype", "cfsf", "heating", "lssf", "spa", "pool", "unitcnt", "year"]
    stdData = StandardScaler().fit_transform(data[numeric_columns])
    stdData = pd.DataFrame(stdData, columns=numeric_columns)
    return stdData

## pca

In [4]:
from sklearn.decomposition import PCA as sklearnPCA
def pca(data):
    tempPCA = sklearnPCA(n_components=8)
    newIndex = tempPCA.fit_transform(data)
    newData = pd.DataFrame(newIndex)
    return newData

## 카테고리 변수 추가

In [ ]:
newData["landuse"] = data2["landuse"]
newData["cluster"] = data2["cluster"]
newData["price"] = data2["price"]

## 예측값 실제값 비교

In [6]:
def predict(model, data):
    pred = model.fittedvalues
    prediction = pd.DataFrame((pred))
    prediction["price"] = data["price"]
    prediction.columns = ["predict", "price"]
    prediction.head(30)